In [1]:
import sys
import Quandl
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/christianjunge/OneDrive/AC297Capstone/AC297_beta/work/Friedman')

In [3]:
import Quandl

In [4]:
from get_beta import get_beta

## Copied in from Haosu's notebook:

In [5]:
#Global Variables
SECTORS = ['Basic Industries', 'Capital Goods', 'Consumer Durables', 'Consumer Non-Durables','Consumer Services', 
           'Energy', 'Finance', 'Health Care', 'Miscellaneous', 'Public Utilities', 'Technology', 'Transportation', 'n/a']
MARKETS = ['NYSE', 'NASDAQ', 'AMEX']

In [6]:
def getStockUniverse(directory = '../../data/', NYSE = True, NASDAQ = True, AMEX = True):
    """
    Reads in the stored stock data, including symbol, Name, Cap, Sector, Industry from NYSE, NASDAQ and AMEX
    
    Returns
    -------
    A pandas frame concatenating all the stock universe from NYSE, NASDAQ and AMEX
    """
    nyse = pd.read_csv(directory+'companylist_nyse.csv')
    nyse['Market'] = 'NYSE'
    nasdaq = pd.read_csv(directory+'companylist_nasdaq.csv')
    nasdaq['Market'] = 'NASDAQ'
    amex = pd.read_csv(directory+'companylist_amex.csv')
    amex['Market'] = 'AMEX'
    
    allstocks = pd.concat([nyse,nasdaq,amex])
    
    # The following parses the market cap into more readable fashion
    # eg. $1.15B = 1150, n/a = -1
    def parseMarketCap(allstocks):
        cap = allstocks['MarketCap'].values
        def computeCap(s):
            if s=='n/a':
                return -1
            elif s[-1:]=='B':
                return float(s[1:-1])*1000
            elif s[-1:]=='M':
                return float(s[1:-1])
        allstocks['MarketCap'] = map(computeCap, cap)
    
    allstocks = allstocks.drop('Unnamed: 8', axis = 1)
    parseMarketCap(allstocks)
    
    allstocks = allstocks.drop_duplicates('Name')    #Drop duplicates, eg. GOOG instead of GOOGL

    return allstocks.sort_values(by = 'MarketCap', ascending = False).reset_index().drop('index', axis=1)

In [7]:
def getSamplePortfolio(stock_universe, n = 10, capThreshold = 2000,
                       sector = None, descendingByCap = False, market = ['NYSE', 'NASDAQ']):
    """
    Get a sample portfolio from the stock universe, preferably a generated one from getStockUniverse.
    With specified parameters.
    
    Parameters
    ----------
    stock_universe : pandas frame with symbols, Cap and Sector
    n : number of samples
    capThreshold : sample should be larger than this threshold, initially set ot 2000
    sector : list of specific sectors to sample from, default to None with no preference
    DescendingByCap: Sample the largest caps, otherwise random sampling.
    market : specific market to sample from, NYSE, NASDAQ and AMEX, default is NYSE and NASDAQ.
    
    Returns
    ------
    A list of symbols
    """
    
    reduced_universe = stock_universe
    
    if capThreshold != None:# and line['MarketCap'].value <capThreshold:
        reduced_universe = reduced_universe[reduced_universe['MarketCap']>=capThreshold]
        
    if sector != None:
        reduced_universe = reduced_universe[map(lambda x: x in sector, reduced_universe['Sector'].values)]
        
    if market != None:
        reduced_universe = reduced_universe[map(lambda x: x in market, reduced_universe['Market'].values)]
    
    if len(reduced_universe)<n:
        print '*******Warning: insufficient candidates, reduce number of samples'
        n = len(reduced_universe)
    
    if descendingByCap:
        # getting rid of redundant quotes, eg. goog and googl
        reduced_universe = reduced_universe.sort_values(by = 'MarketCap', ascending=False)[:n]

    else:
        reduced_universe = reduced_universe.sample(n)
    
    print 'Top 5 rows of selected portfolio:'
    print reduced_universe[['Symbol', 'Name', 'MarketCap', 'Sector', 'industry', 'Market']].head()
    
    return reduced_universe.Symbol.values

In [8]:
def getStocks(symbols, trim_start="2005-01-01", trim_end="2015-12-31"):
    """
    Paramters
    ---------
    symbols: list of string symbols. eg. ['AAPL', 'KCG']
    
    Returns
    -------
    a dictionary of pandas data frames containing all the close prices.
    """
    print symbols
    dfs = {}
    for symbol in symbols:
        flag = False
        #Get data from either Yahoo or Google 
        for source in ['YAHOO/', 'WIKI/', 'GOOG/NYSE_', 'GOOG/NASDAQ_', 'GOOG/AMEX_']:
            try:
                data = Quandl.get(source+symbol,authtoken='c2365v55yoZrWKxbVxwK',
                                  trim_start = trim_start, trim_end = trim_end)
                flag = True
                break
            except:
                pass
        
        if not flag:
            print 'Retrieving ' + symbol + ' unsuccessful. - No symbol. Suggesting:******'
            print Quandl.search(symbol)
            break
            
        else:
            try:
                data = data[['Adjusted Close']]
                data.columns = ['Close']
            except:
                try:
                    data = data[['Close']]
                except:
                    print 'Retrieving ' + symbol + ' unsuccessful. - No close data.'
                    data = None

        dfs[symbol] = data
        
    return dfs

In [9]:
stock_universe = getStockUniverse()

In [10]:
techgiants = getSamplePortfolio(stock_universe, sector = ['Technology'], descendingByCap = True)


Top 5 rows of selected portfolio:
   Symbol                   Name  MarketCap      Sector  \
0    AAPL             Apple Inc.     537330  Technology   
1    GOOG          Alphabet Inc.     485310  Technology   
2    MSFT  Microsoft Corporation     405750  Technology   
4      FB         Facebook, Inc.     307170  Technology   
25   ORCL     Oracle Corporation     155280  Technology   

                                           industry  Market  
0                            Computer Manufacturing  NASDAQ  
1   Computer Software: Programming, Data Processing  NASDAQ  
2           Computer Software: Prepackaged Software  NASDAQ  
4   Computer Software: Programming, Data Processing  NASDAQ  
25          Computer Software: Prepackaged Software    NYSE  


In [11]:
techgiants_p = getStocks(techgiants)

['AAPL' 'GOOG' 'MSFT' 'FB' 'ORCL' 'INTC' 'CSCO' 'IBM' 'TSM' 'DCM']


## It would be better to return the portfolio as a data frame rather than a dictionary of separate data frames.



In [12]:
techgiants_p['AAPL'].values

array([[   4.186585],
       [   4.229582],
       [   4.266626],
       ..., 
       [ 108.153132],
       [ 106.740798],
       [ 104.691918]])

In [13]:
techgiants_p.keys()

['GOOG', 'TSM', 'IBM', 'INTC', 'AAPL', 'CSCO', 'ORCL', 'FB', 'DCM', 'MSFT']

In [14]:
ba = techgiants_p['AAPL']
ba = ba.Close
ba.index

DatetimeIndex(['2005-01-03', '2005-01-04', '2005-01-05', '2005-01-06',
               '2005-01-07', '2005-01-10', '2005-01-11', '2005-01-12',
               '2005-01-13', '2005-01-14',
               ...
               '2015-12-17', '2015-12-18', '2015-12-21', '2015-12-22',
               '2015-12-23', '2015-12-24', '2015-12-28', '2015-12-29',
               '2015-12-30', '2015-12-31'],
              dtype='datetime64[ns]', name=u'Date', length=2769, freq=None)

In [15]:
for key in techgiants_p.keys():
    print len(techgiants_p[key])

2769
2784
2769
2769
2769
2769
2769
911
2784
2769


### This is a problem in this method: the returns are all different sizes.  This will make these hard to work with.

### For the purpose of this development, I'll just use the techgiants that have matching length as a market to hedge against.  We will have to build in ways to deal with missing data.  

In [16]:
market_names=[]
for key in techgiants_p.keys():
    if len(techgiants_p[key])==2769: 
        market_names.append(key)

In [17]:
market_names

['GOOG', 'IBM', 'INTC', 'AAPL', 'CSCO', 'ORCL', 'MSFT']

In [18]:
market = np.zeros((len(techgiants_p['AAPL']),len(market_names)))
for i,key in enumerate(market_names):
    market[:,i] = techgiants_p[key].values.flatten()
    

In [19]:
market

array([[ 101.253921,   79.099823,   16.725715, ...,   16.975329,
          12.453647,   20.887241],
       [  97.15301 ,   78.250155,   16.392216, ...,   16.307562,
          12.128609,   20.965354],
       [  96.658507,   78.088317,   16.232715, ...,   16.316349,
          12.165756,   20.918487],
       ..., 
       [ 776.599976,  138.366653,   35.130791, ...,   27.555284,
          36.887906,   56.14687 ],
       [ 771.      ,  137.9311  ,   34.684721, ...,   27.317139,
          36.768396,   55.908583],
       [ 758.880005,  136.22849 ,   34.149431, ...,   26.949999,
          36.379998,   55.084498]])

### I'll just use a simple unweighted mean of these stocks as an example hedging market.

In [20]:
market = np.mean(market, axis=1)

In [21]:
market = pd.DataFrame(market,index = techgiants_p['AAPL'].Close.index)

In [22]:
portfolio = pd.DataFrame(techgiants_p['AAPL'].values.flatten(), index = techgiants_p['AAPL'].Close.index)

In [23]:
beta = get_beta(portfolio, market)

In [24]:
betas = beta[1]

### Notice that betas is shorter than the market and portfolios by the length of the regression window.  So below, I pass in the time after the 60 day window.  

In [25]:
len(market.ix[60:,:])

2709

In [26]:
betas = pd.DataFrame(betas, index = portfolio.index[60:])

In [27]:
len(betas)

2709

In [28]:
def beta_hedging(betas, market, portfolio):
    """
    Performs beta-hedging, given a portfolio, a market, and beta forecasts.
    
    Parameters:
    -----------
    betas: Pandas Series
        Beta forecasts matching length and frequency of the provided market and portfolio.
    market: Pandas Series
        Asset values of the market that is used to hedge the portfolio.
    portfolio: Pandas Series
        Asset values of the portfolio that is being traded.
    
    Returns:
    --------
    returns: Pandas Series
        Daily returns of the hedged portfolio
    """
    assert len(betas) == len(market) == len(portfolio), "Inputs must be same size"
    
    betasvals = betas.values.flatten()
    marketvals = market.values.flatten()
    portfoliovals = portfolio.values.flatten()
    
    length = len(betas)
    datestamps = betas.index
    returns = np.zeros(length-1)
    
    for day in range(length - 1):
        # Today and tomorrow's prices
        portfolio_buyprice = portfoliovals[day]
        portfolio_sellprice = portfoliovals[day+1]
        market_buyprice = marketvals[day]
        market_sellprice = marketvals[day+1]
        
        # Ratio used to correct for difference in price between market and portfolio
        price_ratio = portfolio_buyprice/market_buyprice
        
        # Daily returns of the portfolio and the hedge
        portfolio_price_change = portfolio_sellprice - portfolio_buyprice
        hedge_price_change = betasvals[i]*price_ratio*(market_sellprice-market_buyprice)
        
        # Daily returns are the difference between the portfolio and the hedge
        returns[day] = portfolio_price_change - hedge_price_change
    
    return pd.DataFrame(returns, datestamps[1:])
    

### Now we can compare the returns from the hedged and unhedged portfolio

In [29]:
hedged_returns = beta_hedging(betas, market.ix[60:,:], portfolio.ix[60:,:])

In [30]:
portfolio_returns = np.diff(portfolio.values.flatten())

### Total returns

In [31]:
np.sum(hedged_returns)

0    109.712116
dtype: float64

In [32]:
np.sum(portfolio_returns)

100.50533299999999

### Volatility of returns

In [33]:
np.std(hedged_returns)

0    1.035375
dtype: float64

In [34]:
np.std(portfolio_returns)

0.96525082598780876

In [35]:
betas[0][0]

-0.23518339950402686

### Hedging a random portfolio against a specific sector

In [140]:
a = pd.DataFrame()

In [144]:
len(b.columns)

1

In [143]:
b=pd.DataFrame(np.array([1,2,3]))

In [120]:
a['yourmom']=b

In [124]:
b=pd.Series(b)
b

TypeError: 'numpy.int64' object is not iterable

In [148]:
trim_start = '2007-01-01'
trim_end='2007-12-31'
portfolio_df = pd.DataFrame()

while(len(portfolio_df.columns)<10):
    potential_stock = getSamplePortfolio(stock_universe, n=1);
    potential_stock_df = getStocks(potential_stock, trim_start=trim_start, trim_end=trim_end);
    print
    print 
    print
    try:
        potential_stock_series = potential_stock_df[potential_stock[0]]
        if len(potential_stock_series)==251: 
            portfolio_df[potential_stock[0]] = potential_stock_series['Close']
    except KeyError:
        pass
portfolio_df

Top 5 rows of selected portfolio:
     Symbol                             Name  MarketCap  \
1348    SAM  Boston Beer Company, Inc. (The)       2420   

                     Sector                             industry Market  
1348  Consumer Non-Durables  Beverages (Production/Distribution)   NYSE  
['SAM']



Top 5 rows of selected portfolio:
     Symbol                      Name  MarketCap       Sector  \
1349   IPXL  Impax Laboratories, Inc.       2420  Health Care   

                   industry  Market  
1349  Major Pharmaceuticals  NASDAQ  
['IPXL']



Top 5 rows of selected portfolio:
    Symbol                     Name  MarketCap   Sector  \
354    MCO  Moody&#39;s Corporation      17520  Finance   

                       industry Market  
354  Finance: Consumer Services   NYSE  
['MCO']



Top 5 rows of selected portfolio:
     Symbol                            Name  MarketCap       Sector  \
1354   RARE  Ultragenyx Pharmaceutical Inc.       2420  Health Care   

            

,SAM,IPXL,MCO,OMC,LPX,DRE,LPL,PPS,THS,CSL
Date,,,,,,,,,,
2007-01-03,36.160000,9.80,61.798170,41.948821,20.681605,24.123975,15.88,33.264671,31.379999,34.243435
2007-01-04,35.939999,9.80,61.349973,42.122350,20.606124,24.218811,15.77,33.389421,31.980000,34.079402
2007-01-05,35.509998,9.80,60.444796,42.242169,20.596689,23.857248,15.34,32.787678,30.600000,33.980123
2007-01-08,35.450001,9.80,60.427223,42.444623,20.757085,23.916519,15.27,32.670265,30.680000,33.945590
2007-01-09,35.290001,9.80,59.935082,42.605758,20.908045,24.236592,15.18,33.470143,30.750000,33.980123
2007-01-10,35.320000,9.80,60.567829,42.952816,20.728779,24.538884,15.20,34.255345,30.809999,33.889475
2007-01-11,35.189999,9.80,61.332399,43.126345,21.313752,24.710773,15.16,34.328727,31.469999,34.545594
2007-01-12,34.950001,9.80,60.963299,43.093294,21.426972,24.746338,15.35,34.453480,31.629999,34.489480
2007-01-16,34.099998,9.80,60.954509,42.969343,21.502454,24.912300,15.65,34.710321,30.530001,34.403146


In [174]:
def equalize_by_top_row(df):
    top_row_sum = np.sum(df.iloc[0,:])
    arr_to_return = np.zeros(df.shape)
    for c in range(len(df.columns)):
        for r in range(df.shape[0]):
            arr_to_return[r,c] = df.iloc[r,c]*top_row_sum/df.iloc[0,c]
    return pd.DataFrame(arr_to_return,index=df.index, columns=df.columns)

In [186]:
market = getStocks(['SPY'],trim_start=trim_start,trim_end=trim_end)['SPY']

['SPY']


In [185]:
portfolio_df2 = equalize_by_top_row(portfolio_df)

In [190]:
portfolio_df2

,SAM,IPXL,MCO,OMC,LPX,DRE,LPL,PPS,THS,CSL
Date,,,,,,,,,,
2007-01-03,309.280676,309.280676,309.280676,309.280676,309.280676,309.280676,309.280676,309.280676,309.280676,309.280676
2007-01-04,307.398982,309.280676,307.037589,310.560072,308.151904,310.496518,307.138304,310.440548,315.194274,307.799159
2007-01-05,303.721133,309.280676,302.507459,311.443475,308.010809,305.861111,298.763575,304.845799,301.593021,306.902488
2007-01-08,303.207972,309.280676,302.419511,312.936130,310.409433,306.620993,297.400247,303.754143,302.381499,306.590592
2007-01-09,301.839474,309.280676,299.956498,314.124148,312.666947,310.724479,295.647397,311.191067,303.071418,306.902488
2007-01-10,302.096059,309.280676,303.123201,316.682940,309.986134,314.600004,296.036919,318.491539,303.662767,306.083772
2007-01-11,300.984145,309.280676,306.949637,317.962336,318.734045,316.803702,295.257875,319.173813,310.167714,312.009723
2007-01-12,298.931414,309.280676,305.102406,317.718658,320.427181,317.259662,298.958336,320.333713,311.744671,311.502911
2007-01-16,291.661240,309.280676,305.058414,316.804791,321.555968,319.387372,304.801170,322.721711,300.903112,310.723158


In [187]:
market

,Close
Date,
2007-01-03,141.37
2007-01-04,141.67
2007-01-05,140.54
2007-01-08,141.19
2007-01-09,141.07
2007-01-10,141.54
2007-01-11,142.16
2007-01-12,143.24
2007-01-16,142.96


In [188]:
portfolio_df2.apply(np.mean,axis=1)

Date
2007-01-03    309.280676
2007-01-04    309.349803
2007-01-05    305.292955
2007-01-08    305.500120
2007-01-09    306.540459
2007-01-10    308.004401
2007-01-11    310.732367
2007-01-12    311.125963
2007-01-16    310.289761
2007-01-17    308.353129
2007-01-18    306.936231
2007-01-19    310.533339
2007-01-22    309.124957
2007-01-23    311.474544
2007-01-24    315.154418
2007-01-25    312.470227
2007-01-26    312.346078
2007-01-29    312.013620
2007-01-30    312.667716
2007-01-31    313.292060
2007-02-01    316.474287
2007-02-02    317.477085
2007-02-05    317.271229
2007-02-06    320.948699
2007-02-07    322.916396
2007-02-08    325.264239
2007-02-09    323.774637
2007-02-12    322.391380
2007-02-13    322.850700
2007-02-14    322.224714
                 ...    
2007-11-16    295.362993
2007-11-19    287.149839
2007-11-20    285.269797
2007-11-21    277.203882
2007-11-23    281.425858
2007-11-26    276.033988
2007-11-27    280.051445
2007-11-28    285.372424
2007-11-29    285.76

In [189]:
equalize_by_top_row(market)

,Close
Date,
2007-01-03,141.37
2007-01-04,141.67
2007-01-05,140.54
2007-01-08,141.19
2007-01-09,141.07
2007-01-10,141.54
2007-01-11,142.16
2007-01-12,143.24
2007-01-16,142.96
